In [2]:
import sympy as sp
import numpy as np
from sympy import *
from IPython.display import Math, Latex

## Developing the state space model

In [3]:
pos_y, vel_y, acc_y, pos_z, vel_z, acc_z, theta_roll, omega_roll, alpha_roll = sp.symbols('y, \dot{y}, \ddot{y}, z, \dot{z}, \ddot{z}, \phi, \dot{\phi}, \ddot{\phi}')
mass, I_x = sp.symbols('m, I_{x}')
g = sp.symbols('g')
u1, u2 = sp.symbols('u_{1}, u_{2}')

acc_y = -u1*sp.sin(theta_roll)/mass
acc_z = u1*sp.cos(theta_roll)/mass - g
alpha_roll = u2/I_x

#display(pos_y, vel_y, acc_y, pos_z, vel_z, acc_z, theta_roll, omega_roll, alpha_roll, mass, I_x, g, u1, u1)
#display(acc_y)
display(Math(r'\ddot{{y}} = {}'.format(sp.latex(acc_y))))
display(Math(r'\ddot{{z}} = {}'.format(sp.latex(acc_z))))
display(Math(r'\ddot{{\phi}} = {}'.format(sp.latex(alpha_roll))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
x1, x2, x3, x4, x5, x6 = sp.symbols('x_{1}, x_{2}, x_{3}, x_{4}, x_{5}, x_{6}')

state_vector_lhs = sp.Matrix([[x1],[x2],[x3],[x4],[x5],[x6]])
state_vector_rhs = sp.Matrix([[pos_y],[pos_z],[theta_roll],[vel_y],[vel_z],[omega_roll]])

display(Math(r'{} = {}'.format(sp.latex(state_vector_lhs), sp.latex(state_vector_rhs))))

<IPython.core.display.Math object>

In [5]:
x1d, x2d, x3d, x4d, x5d, x6d = sp.symbols('\dot{x}_{1}, \dot{x}_{2}, \dot{x}_{3}, \dot{x}_{4}, \dot{x}_{5}, \dot{x}_{6}')
acc_y_ss, acc_z_ss, alpha_roll_ss = sp.symbols('\ddot{y}, \ddot{z}, \ddot{\phi}')
state_vector_lhs_2 = sp.Matrix([[x1d],[x2d],[x3d],[x4d],[x5d],[x6d]])
state_vector_rhs_2 = sp.Matrix([[vel_y],[vel_z],[omega_roll],[acc_y_ss],[acc_z_ss],[alpha_roll_ss]])

display(Math(r'{} = {}'.format(sp.latex(state_vector_lhs_2), sp.latex(state_vector_rhs_2))))

<IPython.core.display.Math object>

In [6]:
state_vector_lhs_3 = sp.Matrix([[x1d],[x2d],[x3d],[x4d],[x5d],[x6d]])
state_vector_rhs_3 = sp.Matrix([[x4],[x5],[x6],[acc_y],[acc_z],[alpha_roll]])

display(Math(r'{} = {}'.format(sp.latex(state_vector_lhs_3), sp.latex(state_vector_rhs_3))))

<IPython.core.display.Math object>

### Linearizing the non-linear model.

Since most of the UAV operation is near the equilibrium hover state, the roll angle $\phi = 0$ is substitued in the non-linear model. At such a small angle, $\sin(\phi) = \phi$ and $\cos(\phi) = 1$ following the first order Taylor series expansion. Therefore, the modified $y$ and $z$ accelerations become:

In [7]:
acc_y = -u1*theta_roll/mass
acc_z = u1/mass - g

display(Math(r'\ddot{{y}} = {}'.format(sp.latex(acc_y))))
display(Math(r'\ddot{{z}} = {}'.format(sp.latex(acc_z))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Also, at the equilibrium hover state, the net force (in the z-direction) acting on the UAV balances its weight. Therefore, $u_{1} = mg$. The modified $y$ acceleration becomes:

In [8]:
acc_y = -g*theta_roll

display(Math(r'\ddot{{y}} = {}'.format(sp.latex(acc_y))))

<IPython.core.display.Math object>

The linearized state space model then becomes:

In [9]:
state_vector_lhs_4 = sp.Matrix([[x1d],[x2d],[x3d],[x4d],[x5d],[x6d]])
state_vector_rhs_4 = sp.Matrix([[x4],[x5],[x6],[acc_y],[acc_z],[alpha_roll]])

display(Math(r'{} = {} = {}'.format(sp.latex(state_vector_lhs_4), sp.latex(state_vector_rhs_2), sp.latex(state_vector_rhs_4))))

<IPython.core.display.Math object>

## Developing the control equations

The variables to be controlled in the dynamic model are position, velocity, and accelerations (linear and angular) in order for the UAV to follow the desired trajectory.

1. Need for position control: For a UAV to reach a target, it is important that the discrete position coordinates that make up the defined/undefined trajectory be precisely followed and the UAV reaches the target. Position control is needed for this purpose.
2. Need for velocity control: When the UAV traverses along a defined/undefined trajectory, it may be needed that it do so with a specified velocity (exact or within a specified range), and when the UAV is about to reach the target, its velocity needs to be decreased gradually to 0. Velocity control is needed for this purpose.
3. Need for acceleration control: When the UAV traverses along a defined/undefined trajectory, it may be needed that it do so with a specified acceleration (exact or within a specified range), so that abrupt increase/decrease in velocities do not occur. Acceleration control is needed for this purpose.

The input variables controlling the position, velocity, and acceleration are thrust ($u_{1}$) and torque ($u_{2}$). From the linearized state space model, the equations for $u_{1}$ and $u_{2}$ are:

In [10]:
u1 = mass*g + mass*acc_z_ss
u2 = I_x*alpha_roll_ss

display(Math(r'u_{{1}} = {}'.format(sp.latex(u1))))
display(Math(r'u_{{2}} = {}'.format(sp.latex(u2))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The commands for the required thrust ($u_{1}$) and torque ($u_{2}$) depend on the commands for the required z-acceleration ($\ddot{z}$) and roll-acceleration ($\ddot{\phi}$), i.e.,

In [11]:
acc_y_com, acc_z_com, alpha_roll_com = sp.symbols('\ddot{y}_{command}, \ddot{z}_{command}, \ddot{\phi}_{command}')

u1_com = mass*g + mass*acc_z_com
u2_com = I_x*alpha_roll_com

display(Math(r'u_{{1}} = {}'.format(sp.latex(u1_com))))
display(Math(r'u_{{2}} = {}'.format(sp.latex(u2_com))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The commands for z-acceleration and roll-acceleration, in turn, come as outputs from their respective PID controllers, i.e.,

In [12]:
pos_y_tar, vel_y_tar, acc_y_tar, pos_z_tar, vel_z_tar, acc_z_tar, theta_roll_tar, omega_roll_tar, alpha_roll_tar = sp.symbols('y_{target}, \dot{y}_{target}, \ddot{y}_{target}, z_{target}, \dot{z}_{target}, \ddot{z}_{target}, {\phi}_{target}, \dot{\phi}_{target}, \ddot{\phi}_{target}')
k_p_y, k_d_y, k_p_z, k_d_z, k_p_phi, k_d_phi = sp.symbols('K_{py}, K_{dy}, K_{pz}, K_{dz}, K_{p{\phi}}, K_{d{\phi}}')

acc_z_com = acc_z_tar + k_d_z*(vel_z_tar - vel_z) + k_p_z*(pos_z_tar - pos_z)
alpha_roll_com = alpha_roll_tar + k_d_phi*(omega_roll_tar - omega_roll) + k_p_phi*(theta_roll_tar - theta_roll)

display(Math(r'\ddot{{z}}_{{command}} = {}'.format(sp.latex(acc_z_com))))
display(Math(r'\ddot{{\phi}}_{{command}} = {}'.format(sp.latex(alpha_roll_com))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The modified commands for the required thrust ($u_{1}$) and torque ($u_{2}$) become:

In [13]:
u1_com = mass*g + mass*acc_z_com
u2_com = I_x*alpha_roll_com

display(Math(r'u_{{1}} = {}'.format(sp.latex(u1_com))))
display(Math(r'u_{{2}} = {}'.format(sp.latex(u2_com))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Solving for the unknown $\phi_{target}$

The unknown root variable in the equation for the required torque ($u_{2}$) is $\phi_{target}$. This can be obtained from the equation of the y-acceleration as:

In [32]:
acc_y = -g*theta_roll

display(Math(r'\ddot{{y}} = {}'.format(sp.latex(acc_y))))
display(Math(r'\ddot{{y}}_{{command}} = -\phi_{{command}}g'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The command for y-acceleration comes as output from its PID controller, i.e.,

In [33]:
acc_y_com = acc_y_tar + k_d_y*(vel_y_tar - vel_y) + k_p_y*(pos_y_tar - pos_y)
display(Math(r'\ddot{{y}}_{{command}} = {}'.format(sp.latex(acc_y_com))))

<IPython.core.display.Math object>

Therefore, the commanded roll angle becomes:

In [34]:
theta_roll_com = sp.symbols('\phi_{command}')
theta_roll_com = -(1/g)*acc_y_com

display(Math(r'\phi_{{command}} = {}'.format(sp.latex(theta_roll_com))))

<IPython.core.display.Math object>

However, unlike the z-acceleration and the roll-acceleration, since the roll angle cannot be commanded directly, $\phi_{command}$ will be considered equal to $\phi_{target}$. Therefore, the unknown variable $\phi_{target}$ becomes:

In [37]:
theta_roll_tar = -(1/g)*acc_y_com

display(Math(r'\phi_{{target}} = {}'.format(sp.latex(theta_roll_tar))))

<IPython.core.display.Math object>

## Refining the control equations for y, z, and roll acceleration commands

Rewriting the command equations for the y-acceleration, z-acceleration, and roll-acceleration:

In [38]:
display(Math(r'\ddot{{y}}_{{command}} = {}'.format(sp.latex(acc_y_com))))
display(Math(r'\ddot{{z}}_{{command}} = {}'.format(sp.latex(acc_z_com))))
display(Math(r'\ddot{{\phi}}_{{command}} = {}'.format(sp.latex(alpha_roll_com))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Refining the control equations: y-acceleration command

#### Tuning parameters:
1. $K_{py}$ = Proportional gain applied on y-position error for y-velocity correction <br>
2. $K_{p\dot{y}}$ = Proportional gain applied on y-velocity error for y-acceleration correction <br>
3. $K_{d\dot{y}}$ = Derivative gain applied on y-acceleration error for y-acceleration correction <br>
</br>
The refined control equations are:
</br>
$1. \; \dot{y}_{target} = K_{py}(y_{target} - y)$ <br>
</br>
$\ddot{y}_{target} = K_{p\dot{y}}(\dot{y}_{target} - \dot{y}) + K_{d\dot{y}}(\ddot{y}_{target} - \ddot{y})$ <br>
$\ddot{y}_{command} = \ddot{y}_{target}$ <br>
</br>
$2. \; \ddot{y}_{command} = K_{p\dot{y}}(\dot{y}_{target} - \dot{y}) + K_{d\dot{y}}(\ddot{y}_{target} - \ddot{y})$

## Refining the control equations: z-acceleration command

#### Tuning parameters:
1. $K_{pz}$ = Proportional gain applied on z-position error for z-velocity correction <br>
2. $K_{p\dot{z}}$ = Proportional gain applied on z-velocity error for z-acceleration correction <br>
3. $K_{d\dot{z}}$ = Derivative gain applied on z-acceleration error for z-acceleration correction <br>
</br>
The refined control equations are:
</br>
$1. \; \dot{z}_{target} = K_{pz}(z_{target} - z)$ <br>
</br>
$\ddot{z}_{target} = K_{p\dot{z}}(\dot{z}_{target} - \dot{z}) + K_{d\dot{z}}(\ddot{z}_{target} - \ddot{z})$ <br>
$\ddot{z}_{command} = \ddot{z}_{target}$ <br>
</br>
$2. \; \ddot{z}_{command} = K_{p\dot{z}}(\dot{z}_{target} - \dot{z}) + K_{d\dot{z}}(\ddot{z}_{target} - \ddot{z})$

## Solving for the unknown $\phi_{target}$

$\phi_{target} = -(1/g)(\ddot{y}_{command})$

## Refining the control equations: roll-acceleration command

#### Tuning parameters:
1. $K_{p\phi}$ = Proportional gain applied on roll-angle error for roll-rate correction <br>
2. $K_{ff\phi}$ = Feed-forward gain applied on the target roll-angle <br>
3. $K_{p\dot{\phi}}$ = Proportional gain applied on roll-rate error for roll-acceleration correction <br>
4. $K_{d\dot{\phi}}$ = Derivative gain applied on roll-acceleration error for roll-acceleration correction <br>
</br>
The refined control equations are:
</br>
$1. \; \dot{\phi}_{target} = K_{p\phi}(\phi_{target} - \phi) + K_{ff\phi}(\phi_{target})$ <br>
</br>
$\ddot{\phi}_{target} = K_{p\dot{\phi}}(\dot{\phi}_{target} - \dot{\phi}) + K_{d\dot{\phi}}(\ddot{\phi}_{target} - \ddot{\phi})$ <br>
$\ddot{\phi}_{command} = \ddot{\phi}_{target}$ <br>
</br>
$2. \; \ddot{\phi}_{command} = K_{p\dot{\phi}}(\dot{\phi}_{target} - \dot{\phi}) + K_{d\dot{\phi}}(\ddot{\phi}_{target} - \ddot{\phi})$

Therefore, the commands for the required thrust ($u_{1}$) and torque ($u_{2}$) become: <br>
</br>
$u_{1} = m(\ddot{z}_{command} + g)$ <br>
$u_{2} = I_{x}\ddot{\phi}_{command}$